In [1]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import KFold
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score


%matplotlib inline
%config InlineBackend.figure_formats = {'png', 'retina'}
pd.options.mode.chained_assignment = None  # default='warn'?

# data_key DataFrame
data_key = pd.read_csv('key.csv')

# data_train DataFrame
data_train = pd.read_csv('train.csv')

# data_weather DataFrame
data_weather = pd.read_csv('weather.csv')

rain_text = ['FC', 'TS', 'GR', 'RA', 'DZ', 'SN', 'SG', 'GS', 'PL', 'IC', 'FG', 'BR', 'UP', 'FG+']
other_text = ['HZ', 'FU', 'VA', 'DU', 'DS', 'PO', 'SA', 'SS', 'PY', 'SQ', 'DR', 'SH', 'FZ', 'MI', 'PR', 'BC', 'BL', 'VC' ]

data_weather['codesum'].replace("+", "")
a = []
for i in range(len(data_weather['codesum'])):
    a.append(data_weather['codesum'].values[i].split(" "))
    for i_text in a[i]:
        if len(i_text) == 4:
            a[i].append(i_text[:2])
            a[i].append(i_text[2:])
            
data_weather["nothing"] = 1
data_weather["rain"] = 0
data_weather["other"] = 0
b = -1
for ls in a:
    b += 1
    for text in ls:
        if text in rain_text:
            data_weather.loc[b, 'rain'] = 1
            data_weather.loc[b, 'nothing'] = 0
        elif text in other_text:
            data_weather.loc[b,'other'] = 1
            data_weather.loc[b, 'nothing'] = 0      
# 모든 데이터 Merge
df = pd.merge(data_weather, data_key)

station_nbr = df['station_nbr']
df.drop('station_nbr', axis=1, inplace=True)
df['station_nbr'] = station_nbr

df = pd.merge(df, data_train)

# T 값 처리 하기. Remained Subject = > 'M' and '-'
df['snowfall'][df['snowfall'] == '  T'] = 0.05
df['preciptotal'][df['preciptotal'] == '  T'] = 0.005

# 주말과 주중 구분 작업 하기
df['date'] = pd.to_datetime(df['date'])
df['week7'] = df['date'].dt.dayofweek
df['weekend'] = 0
df.loc[df['week7'] == 5, 'weekend'] = 1
df.loc[df['week7'] == 6, 'weekend'] = 1

df1 = df[df['station_nbr'] == 1]; df11 = df[df['station_nbr'] == 11]
df2 = df[df['station_nbr'] == 2]; df12 = df[df['station_nbr'] == 12]
df3 = df[df['station_nbr'] == 3]; df13 = df[df['station_nbr'] == 13]
df4 = df[df['station_nbr'] == 4]; df14 = df[df['station_nbr'] == 14]
df5 = df[df['station_nbr'] == 5]; df15 = df[df['station_nbr'] == 15]
df6 = df[df['station_nbr'] == 6]; df16 = df[df['station_nbr'] == 16]
df7 = df[df['station_nbr'] == 7]; df17 = df[df['station_nbr'] == 17]
df8 = df[df['station_nbr'] == 8]; df18 = df[df['station_nbr'] == 18]
df9 = df[df['station_nbr'] == 9]; df19 = df[df['station_nbr'] == 19]
df10 = df[df['station_nbr'] == 10]; df20 = df[df['station_nbr'] == 20]

In [5]:
df8 = df8.apply(pd.to_numeric, errors = 'coerce')
df8.describe().iloc[:, :14]
# 없는 Column = codesum, station_nbr, date, sealevel, station_nbr, store_nbr, sunrise, sunset, depart 

,date,tmax,tmin,tavg,depart,dewpoint,wetbulb,heat,cool,sunrise,sunset,codesum,snowfall,preciptotal
count,1.091130e+05,107559.000000,107781.000000,57276.000000,0.0,103896.000000,102342.000000,57276.000000,57276.000000,0.0,0.0,0.0,555.0,108558.000000
mean,1.368902e+18,81.223942,59.992791,72.610465,NaN,58.305556,63.073753,3.505814,11.116279,NaN,NaN,NaN,0.0,0.052168
std,2.573020e+16,14.231255,14.475479,14.743062,NaN,14.789426,12.779986,7.253903,9.316340,NaN,NaN,NaN,0.0,0.209528
min,1.325376e+18,33.000000,19.000000,31.000000,NaN,8.000000,25.000000,0.000000,0.000000,NaN,NaN,NaN,0.0,0.000000
25%,1.346544e+18,72.000000,49.000000,63.000000,NaN,48.000000,54.000000,0.000000,0.000000,NaN,NaN,NaN,0.0,0.000000
50%,1.368490e+18,83.000000,63.000000,76.000000,NaN,63.000000,66.000000,0.000000,11.000000,NaN,NaN,NaN,0.0,0.000000
75%,1.390954e+18,92.000000,73.000000,85.000000,NaN,71.000000,74.000000,2.000000,20.000000,NaN,NaN,NaN,0.0,0.000000
max,1.414714e+18,109.000000,81.000000,93.000000,NaN,77.000000,80.000000,34.000000,28.000000,NaN,NaN,NaN,0.0,3.000000


In [4]:
df8['store_nbr'].unique()

array([39], dtype=int64)

In [6]:
df8_drop_columns = ['codesum', 'station_nbr', 'date', 'sealevel', 'station_nbr', 'store_nbr', 'sunrise', 'sunset', 'depart' ]
df8 = df8.drop(columns = df8_drop_columns)

In [7]:
# np.nan를 포함하고 있는 변수(column)를 찾아서, 그 변수에 mean 값 대입해서 Frame의 모든 Value가 fill 되게 하기.
df8_columns = df8.columns

# Cateogry 값을 포함하는 변수는 np.nan에 mode값으로 대체하고, 나머지 실수 값을 포함한 변수는 np.nan에 mean값으로 대체
for i in df8_columns:
    if (i == 'resultdir'):
        df8[i].fillna(df8[i].mode()[0], inplace=True)
        print(df8[i].mode()[0])
    else:
        df8[i].fillna(df8[i].mean(), inplace=True)

# 이제 모든 변수가 숫자로 표기 되었기 때문에, 가능 함. 
# 상대 습도 추가 #
df8['relative_humility'] = 100*(np.exp((17.625*((df8['dewpoint']-32)/1.8))/(243.04+((df8['dewpoint']-32)/1.8)))/np.exp((17.625*((df8['tavg']-32)/1.8))/(243.04+((df8['tavg']-32)/1.8))))

# 체감온도 계산
df8["windchill"] = 35.74 + 0.6215*df8["tavg"] - 35.75*(df8["avgspeed"]**0.16) + 0.4275*df8["tavg"]*(df8["avgspeed"]**0.16)

df8 = df8[df8['units'] != 0]

17.0


In [10]:
# 'np.log1p(units) ~ tmax + tmin + tavg + depart + dewpoint + wetbulb + heat + cool + sunrise + sunset + codesume + \
# snowfall + preciptotal + stnpressure + sealevel + resultspeed + resultdir + avgspeed + nothing + rain + other + \
# store_nbr + station_nbr + item_nbr'

model_df8 = sm.OLS.from_formula('np.log1p(units) ~ tmax + tmin + tavg + dewpoint + heat + cool + preciptotal + \
resultspeed + resultdir + avgspeed + C(nothing) + C(rain) + C(other) + C(item_nbr) + C(week7) + \
C(weekend) + relative_humility + windchill + 0', data = df8)
result_df8 = model_df8.fit()

print(result_df8.summary())

                            OLS Regression Results                            
Dep. Variable:        np.log1p(units)   R-squared:                       0.321
Model:                            OLS   Adj. R-squared:                  0.313
Method:                 Least Squares   F-statistic:                     40.31
Date:                Wed, 04 Jul 2018   Prob (F-statistic):          2.44e-146
Time:                        15:22:22   Log-Likelihood:                -1746.1
No. Observations:                1982   AIC:                             3540.
Df Residuals:                    1958   BIC:                             3674.
Df Model:                          23                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
C(nothing)[0]          0.0395      0

In [11]:
model_df8 = sm.OLS.from_formula('np.log1p(units) ~ scale(tmax) + scale(tmin) + scale(tavg) + scale(dewpoint) + scale(heat) + scale(cool) + \
scale(preciptotal) + scale(resultspeed) + scale(resultdir) + scale(avgspeed) + C(nothing) + C(rain) + \
C(other) + C(item_nbr) + C(week7) + C(weekend) + scale(relative_humility) + scale(windchill) + 0', data = df8)
result_df8 = model_df8.fit()

print(result_df8.summary())

                            OLS Regression Results                            
Dep. Variable:        np.log1p(units)   R-squared:                       0.321
Model:                            OLS   Adj. R-squared:                  0.313
Method:                 Least Squares   F-statistic:                     40.31
Date:                Wed, 04 Jul 2018   Prob (F-statistic):          2.44e-146
Time:                        15:24:05   Log-Likelihood:                -1746.1
No. Observations:                1982   AIC:                             3540.
Df Residuals:                    1958   BIC:                             3674.
Df Model:                          23                                         
Covariance Type:            nonrobust                                         
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
C(nothing)[0]           

In [12]:
anova_result_df8 = sm.stats.anova_lm(result_df8).sort_values(by=['PR(>F)'], ascending = False)
anova_result_df8[anova_result_df8['PR(>F)'] <= 0.05]

C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1821: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


,df,sum_sq,mean_sq,F,PR(>F)
scale(windchill),1.0,1.626627,1.626627,4.712808,3.005849e-02
scale(dewpoint),1.0,1.812359,1.812359,5.250929,2.204079e-02
scale(resultspeed),1.0,1.964597,1.964597,5.692004,1.713693e-02
scale(cool),1.0,2.742647,2.742647,7.946241,4.867190e-03
C(item_nbr),3.0,306.269576,102.089859,295.783816,2.404581e-158
C(nothing),2.0,7264.058886,3632.029443,10523.038651,0.000000e+00


In [14]:
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(df8.values, i) for i in range(df8.shape[1])]
vif["features"] = df8.columns
vif = vif.sort_values("VIF Factor").reset_index(drop=True)
vif
# 10순위까지 겹치는것만 쓴다
# item_nbr, nothing, 

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1386: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.centered_tss


,VIF Factor,features
0,1.044993,item_nbr
1,1.045768,units
2,1.195192,preciptotal
3,1.244939,resultdir
4,1.524272,other
5,2.255102,stnpressure
6,2.631666,week7
7,2.638345,weekend
8,5.573153,rain
9,6.217114,nothing


In [15]:
# resultdir, preciptotal, C(rain), C(week7), C(item_nbr)
model_df8 = sm.OLS.from_formula('np.log1p(units) ~ C(item_nbr) + C(nothing) + 0', data = df8)
result_df8 = model_df8.fit()

print(result_df8.summary())

                            OLS Regression Results                            
Dep. Variable:        np.log1p(units)   R-squared:                       0.309
Model:                            OLS   Adj. R-squared:                  0.308
Method:                 Least Squares   F-statistic:                     221.2
Date:                Wed, 04 Jul 2018   Prob (F-statistic):          4.89e-157
Time:                        15:26:31   Log-Likelihood:                -1763.6
No. Observations:                1982   AIC:                             3537.
Df Residuals:                    1977   BIC:                             3565.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
C(item_nbr)[9]       1.7396      0.023  

In [ ]:
# resultdir, preciptotal, C(rain), C(week8), item_nbr, weekend
X8 = df8[['item_nbr', 'nothing']]
y8 = df8['units']
model8 = LinearRegression()

cv8 = KFold(n_splits=10, shuffle=True, random_state=0)

cross_val_score(model8, X8, y8, scoring="r2", cv=cv8)